In [1]:
import numpy as np

In [2]:
def create_tensor(dims):
    '''
    Constructs a tensor of random integers, where each index has dimensions given by the argument dims (tuple)
    '''
    return np.random.random_sample(dims)

def make_diag(s, a, vh):
    '''
    converts vector of singular values into diagonal matrix of correct dimensions
    '''
    sigma = np.zeros((a.shape[0], vh.shape[0]))
    sigma[:s.shape[0], :s.shape[0]] = np.diag(s)
    return sigma

In [3]:
### left canonical ###
def left_sweep(mps):
    '''
    Returns a matrix product state (mps) in left-canonical form.
    '''
    a_double_primes = []
    singular_vals = []
#     lams = []
    # first step: svd on mps[0]
    a1pp, s1, vh1 = np.linalg.svd(mps[0]) # a1pp is a1-double-prime
#     print('a1pp', a1pp.shape)
    singular_vals.append(s1)
    s1 = make_diag(s1, a1pp, vh1)
#     lams.append(s1)
    a2p = np.tensordot((s1 @ vh1), mps[1], axes = (1, 1)) # a2-prime
    a2p = np.transpose(a2p, (1, 0, 2)) # physical index first, followed by aux indices
    a_double_primes.append(a1pp)
#     print('lam1', s1.shape)

    # sweep: reshape and perform svd on the tensors, stopping at second-to-last tensor
    for l in range(1, len(mps)-2):
        a2p = a2p.reshape((d * a2p.shape[1], a2p.shape[2]))
        a2pp, s2, vh2 = np.linalg.svd(a2p)
        singular_vals.append(s2)
        s2 = make_diag(s2, a2pp, vh2)
#         lams.append(s2)
        a3p = np.tensordot((s2 @ vh2), mps[l+1], axes = (1, 1))
        a3p = np.transpose(a3p, (1, 0, 2)) # physical index first, followed by aux indices
        a2pp = a2pp.reshape((d, a_double_primes[-1].shape[-1], a3p.shape[1]))
        a_double_primes.append(a2pp)
        a2p = a3p

    # last step: svd on mps[3] (second-to-last tensor)
    a2p = a2p.reshape((d * a2p.shape[1], a2p.shape[2]))
    a2pp, s2, vh2 = np.linalg.svd(a2p)
    singular_vals.append(s2)
    s2 = make_diag(s2, a2pp, vh2)
#     lams.append(s2)
    a3p = np.tensordot((s2 @ vh2), mps[-1], axes = (1, 1))
    a3p = np.transpose(a3p, (1, 0))
    a2pp = a2pp.reshape((d, a_double_primes[-1].shape[-1], a3p.shape[1]))
    a_double_primes.append(a2pp)
    a_double_primes.append(a3p)
    return a_double_primes, singular_vals

In [4]:
def test_left_env(left_canonical, mps):
    kets = left_canonical
    bras = [np.conj(m.T) for m in kets]
    
#     print('original mps')
#     for i in mps:
#         print(i.shape)

#     print('\nleft canonical form')
#     for i in kets:
#         print(i.shape)

    ##################################### test 1 ####################################################
    # contracting left canonical bras and kets
    print('\ntest 1...')

    eprev = np.tensordot(bras[0], kets[0], axes = 1) # first contraction
    print('identity?', np.allclose(eprev, np.eye(*eprev.shape))) # check

    for i in range(1, len(kets)):
        if i < len(kets)-1:
            c = np.tensordot(eprev, kets[i], axes = (-1, 1)) # second contraction 
            eprev = np.tensordot(bras[i], c, axes = 2)
            print('identity?', np.allclose(eprev, np.eye(*eprev.shape))) # check
        else:
            c = np.tensordot(eprev, kets[-1], axes = (-1, 1)) # final contraction 
            E1 = np.tensordot(bras[-1], c, axes = 2)

    E2 = np.tensordot(bras[-1], kets[-1], axes = ([1,0], [0,1])) # treating all the other contractions as identities
    print('Test 1 passed?', np.allclose(E1, E2)) # test E1 and E2 are the same

    ################################## test 2 ########################################################
    # contracting mps and comparing it to contracted canonical form

    bras = [np.conj(m.T) for m in mps]
    kets = mps
    print('\ntest 2...')

    eprev = np.tensordot(bras[0], kets[0], axes = 1) # first contraction

    for i in range(1, len(kets)):
        if i < len(kets)-1:
            c = np.tensordot(eprev, kets[i], axes = (-1, 1)) # second contraction 
            eprev = np.tensordot(bras[i], c, axes = 2)
        else:
            c = np.tensordot(eprev, kets[-1], axes = (-1, 1)) # final contraction 
            E3 = np.tensordot(bras[-1], c, axes = 2)

    print('Test 2 passed?', np.allclose(E2, E3)) # test E2 and E3 are the same

In [5]:
### testing left sweep ###

D = 12 # aux bonds
d = 2 # physical bonds
mps = [create_tensor((d, D)), create_tensor((d, D, D)), create_tensor((d, D, D)), create_tensor((d, D, D)), \
       create_tensor((d, D, D)), create_tensor((d, D, D)), create_tensor((d, D))] # 7 site mps
left_mps, _ = left_sweep(mps)

test_left_env(left_mps, mps)


test 1...
identity? True
identity? True
identity? True
identity? True
identity? True
identity? True
Test 1 passed? True

test 2...
Test 2 passed? True


In [6]:
def right_sweep(kets):
    '''input mps must already have identity as left environment. i.e. do left sweep on mps before input'''
    # note: bras have dimensions (D2, D1, d) and kets have (d, D1, D2)
    
    # sweep from R->L
    bras = [np.transpose(np.conj(i)) for i in kets]
    ws = [] # note: the ws and wdags are appended in order from right to left i.e. w corresponding to site 1 is ws[-1]
    wdags = []
    r = np.tensordot(bras[-1], kets[-1], axes = 1) # r4
    w, lam, wdag = np.linalg.svd(r) # svd on r4
    ws.append(w); wdags.append(wdag)
    r_p = np.tensordot(np.tensordot(wdag, r, axes = 1), w, axes = 1) # r4-prime

    for i in range(2, len(mps)):
        a_p = np.tensordot(kets[-i], w, axes = 1) # a-prime
        adag_p = np.transpose(np.tensordot(bras[-i], wdag, axes = (0, 0)), (2, 0, 1)) # (a-prime)dagger
        a_pp = np.tensordot(a_p, r_p, axes = 1)
        rminus = np.tensordot(a_pp, adag_p, axes = ([0, 2], [2, 0]))
        w, lam, wdag = np.linalg.svd(rminus)
        ws.append(w); wdags.append(wdag)
        r_p = np.tensordot(np.tensordot(wdag, rminus, axes = 1), w, axes = 1)

    # now use ws and wdags to form new kets that have diagonal right envs
    ws.reverse() # reverse ws and wdags so that they go from left to right i.e. ws[0] corresponds to mps[0]
    wdags.reverse()
    # first step
    new_kets = []
    a_new = np.tensordot(kets[0], ws[0], axes = 1)
    new_kets.append(a_new)
    # loop middle section
    for i in range(1, len(kets)-1):
        a_new = np.tensordot(kets[i], wdags[i-1], axes = (1, 1))
        a_new = np.transpose(a_new, (0, 2, 1))
        a_new = np.tensordot(a_new, ws[i], axes = 1)
        new_kets.append(a_new)
    # last step
    a_new = np.tensordot(kets[-1], wdags[-1], axes = (1, 1))
    new_kets.append(a_new)
    return new_kets

In [15]:
### testing left environment ###

D = 100 # aux bonds
d = 2 # physical bonds
mps = [create_tensor((d, D)), create_tensor((d, D, D)), create_tensor((d, D, D)), create_tensor((d, D, D)), \
       create_tensor((d, D, D)), create_tensor((d, D, D)), \
       create_tensor((d, D, D)), create_tensor((d, D, D)), create_tensor((d, D, D)), \
       create_tensor((d, D, D)), create_tensor((d, D, D)), create_tensor((d, D))] # 12 site mps

left_canonical = right_sweep(left_sweep(mps)[0])

test_left_env(left_canonical, mps)


test 1...
identity? True
identity? True
identity? True
identity? True
identity? True
identity? True
identity? True
identity? True
identity? True
identity? True
identity? True
Test 1 passed? True

test 2...
Test 2 passed? True


In [22]:
### testing  physical indices ###

# step 1: contract mps along auxiliary indices
a = mps[0]
for i in range(len(mps)-1):
    a = np.tensordot(a, mps[i+1], axes = (-1, 1))
#     print(a.shape)
mps_contracted = a
# print('mps shape after aux contraction', mps_contracted.shape)

# step 2: contract left-canonical form of mps along auxiliary indices
a = left_canonical[0]
for i in range(len(left_canonical)-1):
    a = np.tensordot(a, left_canonical[i+1], axes = (-1, 1))
#     print(a.shape)
left_canonical_contracted = a
# print('left-canonical mps shape after aux contraction',left_canonical_contracted.shape)

# step 3: retrieve the physical indices for contracted mps
bit_string = (1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0)
x1 = mps_contracted[bit_string]
# step 4: compare with the physical indices for contract left-canonical mps
x2 = left_canonical_contracted[bit_string]
print('mps', x1)
print('left-canonical', x2)
print('%error', np.abs((x1-x2)*100/x1))

mps 2.6220461707115356e+18
left-canonical 2.6187084880272174e+18
%error 0.12729305538553778
